In [17]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 19, Finished, Available, Finished)

In [18]:
%%sql
CREATE TABLE IF NOT EXISTS Ecommere_Lh.gold_product
(
    Product_ID long ,
    Product_Category string ,
    Product string  ,
    Created_TS timestamp    ,
    Modified_TS timestamp   
)
USING DELTA

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [19]:
max_date = spark.sql("""
    SELECT coalesce(max("Modified_TS"),'1900-01-01') FROM Ecommere_Lh.gold_product
    """).first()[0]

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 21, Finished, Available, Finished)

In [20]:
max_date

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 22, Finished, Available, Finished)

'1900-01-01'

In [21]:
df_bronze = spark.sql("""SELECT distinct
Product_Category    ,
Product
FROM Ecommere_Lh.bronze_sales where modified_TS>'{}'

""".format(max_date))

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 23, Finished, Available, Finished)

In [22]:
max_id = spark.sql("SELECT coalesce(max(Product_ID),0) from Ecommere_LH.gold_product").first()[0]

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 24, Finished, Available, Finished)

In [23]:
max_id

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 25, Finished, Available, Finished)

0

In [24]:
df_final = df_bronze.withColumn("Product_ID", monotonically_increasing_id()+max_id+1)

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 26, Finished, Available, Finished)

In [25]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 27, Finished, Available, Finished)

In [26]:
%%sql
SELECT * FROM ViewProduct limit 10

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 28, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 3 fields>

In [27]:
%%sql
MERGE INTO Ecommere_LH.gold_product as gp
USING ViewProduct as vp
on gp.product = vp.product and gp.Product_Category = vp.Product_Category
WHEN MATCHED THEN
UPDATE SET
gp.Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
INSERT
(
gp.Product_ID   ,
gp.Product_Category ,
gp.Product  ,
gp.Modified_TS  ,
gp.Created_TS
)
VALUES
(
    vp.Product_ID   ,
    vp.Product_Category ,
    vp.Product  ,
    CURRENT_TIMESTAMP(),
    CURRENT_TIMESTAMP()

)

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 29, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [28]:
%%sql
SELECT * FROM Ecommere_Lh.gold_product

StatementMeta(, 17d5d6ab-3cb6-4c80-bb6e-5a60f832d17e, 30, Finished, Available, Finished)

<Spark SQL result set with 42 rows and 5 fields>